In [17]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn import datasets
from ipywidgets import interact
import ipywidgets as widgets
from IPython.core.display import HTML
from IPython.display import clear_output

In [18]:
#Dataset Processing
nba19 = pd.read_csv('https://raw.githubusercontent.com/jeremydumalig/Clustering/master/nba19_traditional.csv')
nba19 = nba19.dropna()
nba19.index = list(range(0, len(nba19)))

factors = ['Player', 'FG%', 'FGA', '3PA', '3P%', 'FTA', 'FT%', 'PTS', 'AST', 'ORB', 'DRB', 'STL', 'BLK']
nba19 = nba19[factors]

for x in factors:
    if x== 'Player' or x == 'FG%' or x == '3P%' or x == 'FT%':
        pass
    else:
        nba19[x] = (nba19[x] - min(nba19[x]))/(max(nba19[x]) - min(nba19[x]))

In [19]:
#K-Means Clustering
k=10

k_model = KMeans(n_clusters=k)
k_model.fit(nba19.drop('Player', axis=1))
labels = k_model.labels_

nba19['K-Means'] = labels

In [20]:
#DBSCAN Clustering
d_model = DBSCAN(eps=0.5, min_samples=5).fit(nba19.drop('Player', axis=1).values)

labels = d_model.labels_
nba19['DBSCAN'] = labels

In [21]:
#Hierarchical Clustering
#R: http://rpubs.com/jdumalig/nbaclustering
h = pd.read_csv('https://raw.githubusercontent.com/jeremydumalig/Clustering/master/hierarchical.csv')

In [22]:
#Merged dataframes
kd = nba19[['Player', 'K-Means', 'DBSCAN']]
nba19 = pd.concat([nba19, h[['Hierarchical']]], axis=1, sort=False)
nba19.sort_values(by='PTS', ascending=False)[['Player', 'K-Means', 'DBSCAN', 'Hierarchical']].head()

,Player,K-Means,DBSCAN,Hierarchical
186,James Harden,3,-1,8
166,Paul George,3,8,8
16,Giannis Antetokounmpo,5,-1,5
140,Joel Embiid,5,-1,5
232,LeBron James,3,8,7


In [23]:
#Display cluster by player
def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )
def clusters(Player):
    indices = nba19[(nba19['Player'] == Player)][['K-Means', 'DBSCAN', 'Hierarchical']].iloc[0]

    k = nba19[(nba19['K-Means'] == indices[0])].sort_values(by='PTS', ascending=False)[['Player', 'K-Means']].head(10)
    d = nba19[(nba19['DBSCAN'] == indices[1])].sort_values(by='PTS', ascending=False)[['Player', 'DBSCAN']].head(10)
    h = nba19[(nba19['Hierarchical'] == indices[2])].sort_values(by='PTS', ascending=False)[['Player', 'Hierarchical']].head(10)

    #Player may not appear in displayed dataframes if not PTS leader
    display(multi_table([k, d, h]))

In [24]:
#Number of players in each cluster by type of clustering
k = pd.DataFrame(nba19['K-Means'].value_counts())
h = pd.DataFrame(nba19['DBSCAN'].value_counts())
d = pd.DataFrame(nba19['Hierarchical'].value_counts())

multi_table([k, d, h])

,K-Means
4,103
9,84
6,64
2,56
1,38
7,36
8,33
3,27
0,21
5,13


In [25]:
interact(clusters, Player=list(nba19.sort_values(by='PTS', ascending=False)['Player']))

#clusters('Stephen Curry')

interactive(children=(Dropdown(description='Player', options=('James Harden', 'Paul George', 'Giannis Antetoko…

<function __main__.clusters>

In [26]:
#Display model + cluster
def cluster_display(Cluster):
  display( nba19[(nba19[cluster_type]) == Cluster].sort_values(by='PTS', ascending=False)[['Player', cluster_type]].head(10) )

def cluster_list(Model):
  clear_output()

  global cluster_type
  cluster_type = Model

  interact(cluster_display, Cluster=list((nba19[Model].value_counts()).index))

interact(cluster_list, Model=['K-Means', 'DBSCAN', "Hierarchical"])

interactive(children=(Dropdown(description='Model', options=('K-Means', 'DBSCAN', 'Hierarchical'), value='K-Me…

<function __main__.cluster_list>

# Notes

For my final project, I decided to attempt to determine positions of NBA players using multiple clustering algorithms.

My goal was to use k-means, DBSCAN, and hierarchical clustering to identify similar clusters between algorithms. I interpreted the results solely from a basketball fan perspective. In other words, I'm just looking to see if the clusters from each algorithm have the same players or not.

For example, if I filtered only for clusters that contain Stephen Curry, the k-means cluster also includes James Harden, Paul George, LeBron James, and Devin Booker. Similarly, the DBSCAN cluster includes all but Harden, but also contains Kawhi Leonard. Lastly, the hierarchical cluster contains only Harden, George, and Russell Westbrook.

Footnote: I implemented hierarchical clustering in R, with my code here: http://rpubs.com/jdumalig/nbaclustering